In [1]:
import os
import torch
import random
import numpy as np
import json
import torch.nn as nn
from predict import predict_from_dataset

In [2]:
def loadmodel(name):
    x = open('model_parameters/'+ name+'.json')
    data = json.load(x)
    model_structure = data['model_structure']
    model_parameters_names = data['model_parameters_names']
    model_parameters_vals_= data['model_parameters_vals']
    model_parameters_vals =[]
    for i in model_parameters_vals_:
        tmp=torch.tensor(i)
        tmp2=torch.rand(tmp.shape)*0.0000001
        tmp3=torch.add(tmp,tmp2)
        model_parameters_vals.append(tmp3.tolist())
    model_parameters_names = [f'[{name.split(".")[0]}].{name.split(".")[1]}'
                                   for name in model_parameters_names.split('|')]
    received_model = nn.Sequential(
    *[eval('nn.' + layer) for layer in model_structure.split('|')]
    )
    for parameters_name, parameters_val in zip(model_parameters_names, model_parameters_vals):
        exec(f'received_model{parameters_name} = nn.Parameter(torch.tensor(parameters_val))')
    for param in received_model.parameters():
        param.requires_grad = False
    return received_model.to('cuda')

In [92]:

def checksame():
    hh=['model_A','model_B','model_C','model_D','model_E','model_F','model_G','model_T1','model_T2','model_T3']
    for i in hh:
        y=torch.load('../../saved_models/'+ i+'.pt').to('cuda')
        x=loadmodel(i)
        random_input = torch.randn(32, 768).to('cuda')
        x.eval()
        x_ = x(random_input)
        y.eval()
        y_= y(random_input)
        print(i)
        if (x_ == y_).all():
            print('Success! Two models have the same output')
        else:
            print('Fail! Please check if any of the parameters are different')
checksame()

model_A
Success! Two models have the same output
model_B
Success! Two models have the same output
model_C
Success! Two models have the same output
model_D
Success! Two models have the same output
model_E
Success! Two models have the same output
model_F
Success! Two models have the same output
model_G
Success! Two models have the same output
model_T1
Success! Two models have the same output
model_T2
Success! Two models have the same output
model_T3
Success! Two models have the same output


In [33]:
def loadmodels(names):
    all=[]
    for i in names:
        all.append(loadmodel(i))
    return all


In [34]:
dataset_path = '../../Datasets/CIFAR10'

In [35]:
def ensemble(models,path,device):
    for i in models:
        
        predict_from_dataset(path,'X',"sb","sb",i,device)

In [36]:
def final(path):
    models= loadmodels(["model_A"])
    y= ensemble(models,dataset_path,'cuda')
    return y

In [37]:
final(dataset_path)

<torch.utils.data.dataloader.DataLoader object at 0x00000187EE2858B0> Sequential(
  (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=768, out_features=128, bias=True)
  (2): GELU()
  (3): Linear(in_features=128, out_features=6, bias=True)
) CNM


100%|██████████| 57/57 [00:55<00:00,  1.03it/s]
